




Implementation of Fischer's Dicriminant Analysis from scratch for dimensionality reduction and applying LDA from sklearn for multi-class classification




In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
import os


In [ ]:
os.chdir('/fmnist')   #changing directory

In [ ]:
train_set = pd.read_csv('fashion-mnist_train.csv')
test_set =  pd.read_csv('fashion-mnist_test.csv')

In [ ]:
# Saving the labels
train_label = train_set['label']    
test_label = test_set['label']

In [ ]:
train_data = train_set.drop(['label'], axis=1)   #labels dropped
test_data = test_set.drop(['label'], axis=1)   #labels dropped


In [ ]:
print(train_label.shape)

(60000,)


In [ ]:
# converting pandas dataframe to numpy arrays
train_arr = np.array(train_data)
test_arr = np.array(test_data)
train_label_arr = np.array(train_label)
test_label_arr = np.array(test_label)

In [ ]:
train_arr.shape

(60000, 784)

In [ ]:
def get_W(train_arr, train_label):

  class_n = []

  Sw = 0
  for j in range(10):
    class_n = []
    for i in range(len(train_arr)):
      # class_1 = list(class_1)
      if(train_label[i]==j):
        class_n.append(train_arr[i])

    class_n = np.array(class_n)
    mean_row_cln = np.mean(class_n, axis=0)
    mean_arr_cln = class_n - mean_row_cln
    sn = np.dot(np.transpose(mean_arr_cln), mean_arr_cln)
    Sw += sn

  X = train_arr-np.mean(train_arr, axis=0)
  St = np.dot(np.transpose(X), X)

  Sb = St-Sw 

  eigenval, eigenvec = np.linalg.eig(np.dot(np.linalg.inv(Sw), Sb))
  W = eigenvec[:, :9]

  return W

In [ ]:
W = get_W(train_arr, train_label)

In [ ]:
Y = np.real(np.dot(train_arr, W))     # Projection of training data on Y using W projection

In [ ]:
Y2 = np.real(np.dot(test_arr, W))     # Projection of testing data on Y2 using W projection

In [ ]:
print(W.shape)
print(Y.shape)
print(Y2.shape)

(784, 9)
(60000, 9)
(10000, 9)


In [ ]:
# for classification

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
clf = LDA()
clf.fit(Y, train_label)     # fitting the reduced training set with the labels


LinearDiscriminantAnalysis()

In [ ]:
pred_FDA = clf.predict(Y2)
print(pred_FDA)

[0 1 2 ... 8 6 2]


In [ ]:
from sklearn.metrics import accuracy_score
score_LF= accuracy_score(test_label, pred_FDA) * 100
print(score_LF)


82.56


In [ ]:
# for class-wise accuracy 
def get_classwise_acc(test_label, pred_FDA):

  clx_count = np.zeros([1, 10])
  clx_acc = np.zeros([1, 10])

  for j in range(10):
    for i in range(len(test_label)):
      if(test_label[i]==j and pred_FDA[i]==j):
        clx_count[0][j]+=1
    clx_acc[0][j] = (clx_count[0][j]/np.where(test_label==j)[0].shape[0])*100

  print("Class-wise accuracy: ")
  for i in range(10):
    print("Class", i, "accuracy: ", clx_acc[0][i], "%")
  
  return clx_acc


In [ ]:
acc_all = get_classwise_acc(test_label, pred_FDA)

Class-wise accuracy: 
Class 0 accuracy:  76.8 %
Class 1 accuracy:  94.0 %
Class 2 accuracy:  71.5 %
Class 3 accuracy:  87.1 %
Class 4 accuracy:  78.8 %
Class 5 accuracy:  89.0 %
Class 6 accuracy:  60.5 %
Class 7 accuracy:  85.5 %
Class 8 accuracy:  91.60000000000001 %
Class 9 accuracy:  90.8 %
